# Deteksi Outlier dengan Local Outlier Factor (LOF)

## Apa itu LOF?

Dalam analisis deteksi anomali, Local Outlier Factor (LOF) adalah sebuah algoritma yang digunakan untuk mengidentifikasi data yang menyimpang dengan menilai seberapa jauh suatu titik data berbeda dari tetangga sekitarnya. Algoritma ini pertama kali diperkenalkan pada tahun 2000 oleh Markus M. Breunig, Hans-Peter Kriegel, Raymond T. Ng, dan Jörg Sander. LOF mengadopsi konsep serupa dengan DBSCAN dan OPTICS, yakni menggunakan "jarak inti" dan "jarak keterjangkauan" untuk memperkirakan kerapatan lokal suatu titik data.

## Tahapan LOF

1. **Menentukan parameter LOF** -- Langkah pertama dalam metode Local Outlier Factor (LOF) adalah menentukan parameter utama, yaitu nilai k dan metrik jarak yang akan digunakan. Nilai k menentukan jumlah tetangga terdekat yang digunakan untuk mengevaluasi kepadatan suatu titik data. Pemilihan k yang terlalu kecil dapat menyebabkan model terlalu sensitif terhadap pencilan lokal, sementara nilai yang terlalu besar dapat menyebabkan anomali sulit terdeteksi.

2. **Menentukan k distance dan tetangga terdekat** -- Setelah parameter ditentukan, langkah berikutnya adalah menghitung k-Distance untuk setiap titik dalam dataset. k-Distance adalah jarak dari suatu titik ke tetangga ke-k terdekatnya. Setelah nilai k-Distance diperoleh, daftar k-Nearest Neighbors (k-NN) ditentukan, yaitu sekumpulan titik yang berada dalam radius k-Distance tersebut. Titik-titik dalam k-NN akan menjadi acuan dalam perhitungan kepadatan lokal pada langkah-langkah selanjutnya.

3. **Menghitung Reachability Distance** -- Pada tahap ini, Reachability Distance dihitung untuk setiap pasangan titik yang berada dalam k-NN. Nilai ini ditentukan dengan mengambil nilai maksimum antara k-Distance dari tetangga dan jarak sebenarnya antara dua titik tersebut. Tujuan dari langkah ini adalah untuk memastikan bahwa titik yang sangat dekat dengan tetangganya tetap memiliki jarak keterjangkauan minimal setara dengan k-Distance dari tetangganya, sehingga distribusi kepadatan data lebih stabil.

4. **Menghitung Local Reachability Density (LRD)** -- Setelah Reachability Distance dihitung, langkah berikutnya adalah menentukan Local Reachability Density (LRD) untuk setiap titik. LRD mengukur kepadatan lokal dengan menghitung kebalikan dari rata-rata Reachability Distance antara titik tersebut dan tetangganya. Semakin kecil nilai LRD, semakin rendah kepadatan lokal suatu titik dibandingkan dengan tetangganya. Titik dengan kepadatan rendah dibandingkan lingkungannya berpotensi menjadi anomali.

5. **Menghitung Local Outlier Factor (LOF)** -- Setelah LRD setiap titik diketahui, nilai Local Outlier Factor (LOF) dihitung dengan membandingkan LRD suatu titik terhadap LRD dari tetangga-tetangganya. Nilai LOF diperoleh dengan menghitung rata-rata rasio LRD tetangga terhadap LRD titik tersebut. Jika LOF mendekati 1, artinya kepadatan titik tersebut sebanding dengan tetangganya dan tidak dianggap sebagai pencilan. Namun, jika LOF jauh lebih besar dari 1, titik tersebut memiliki kepadatan yang jauh lebih rendah dibandingkan dengan lingkungannya dan berpotensi menjadi anomali.

6. **Menentukan Anomali Berdasarkan LOF** -- Tahap terakhir adalah menetapkan ambang batas LOF untuk mengidentifikasi titik-titik yang dianggap sebagai anomali. Biasanya, titik dengan nilai LOF lebih besar dari 1.5 atau 2 dikategorikan sebagai pencilan, tergantung pada sensitivitas yang diinginkan. Dengan demikian, metode LOF dapat secara efektif mendeteksi anomali dalam dataset yang memiliki kepadatan tidak seragam, seperti klaster dengan ukuran berbeda atau data dengan distribusi yang kompleks.

## CONTOH MENGHITUNG MANUAL LOF
Berikut adalah cara singkat menghitung **Local Outlier Factor (LOF)** untuk satu titik data dengan dua fitur:

1. **Data**:  
   Misalkan data berikut:

| ID | Feature1 | Feature2 |
|----|----------|----------|
| 1  | 2.0      | 3.0      |
| 2  | 4.0      | 5.0      |
| 3  | 6.0      | 7.0      |
| 4  | 8.0      | 9.0      |
| 5  | 10.0      | 11.0      |

   Kita akan hitung LOF untuk **titik 3** (6.0, 7.0).

2. **Hitung Jarak Euclidean**:  
   Jarak antara titik 3 dan titik lainnya:
   - Titik 3 ke Titik 2: 1.41
   - Titik 3 ke Titik 4: 1.41

3. **Tentukan k-Tetangga Terdekat** (k=2):  
   Titik 3 memiliki dua tetangga terdekat: titik 2 dan titik 4.

4. **Hitung Reachability Distance**:  
   Reachability distance antara titik 3 dan tetangga:
   - Titik 3 ke Titik 2: 2.83
   - Titik 3 ke Titik 4: 2.83

5. **Hitung Local Reachability Density (LRD)**:  
   $
   \text{LRD}(3) = \frac{1}{\frac{1}{2} \times (2.83 + 2.83)} = 0.35
   $

6. **Hitung LOF**:  
   $
   \text{LOF}(3) = \frac{0.35}{0.35} + \frac{0.35}{0.35} = 2
   $

7. **Interpretasi**:  
   Karena **LOF = 2** (lebih besar dari 1), titik 3 dianggap **outlier**.

LOF mengukur seberapa terisolasi titik dibandingkan tetangganya. Titik dengan LOF lebih besar dari 1 dianggap outlier.

## Contoh implementasi menggunakan sklearn menggunakan data iris

In [2]:
%pip install pymysql
%pip install psycopg2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.4 MB/s eta 0:00:00


In [8]:
import psycopg2
import pymysql
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor
from IPython.display import display, HTML

In [9]:
def get_pg_data():
    conn = psycopg2.connect(
        host="postgresqltest-projectadzin.i.aivencloud.com",
        user="avnadmin",
        password="AVNS_k58rpi0pi31N8ucX-vK",
        database="defaultdb",
        port=16209
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM sepal")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  # Ambil nama kolom
    cursor.close()
    conn.close()

    return pd.DataFrame(data, columns=columns)

def get_mysql_data():
    conn = pymysql.connect(
        host="mysql-386c0068-projectadzin.l.aivencloud.com",
        user="avnadmin",
        password="AVNS_-Zr6_we8osFXobQb3qi",
        database="iris",
        port=16209
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM petal")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  # Ambil nama kolom
    cursor.close()
    conn.close()

    return pd.DataFrame(data, columns=columns)

# Ambil data dari kedua database
df_postgresql = get_pg_data()
df_mysql = get_mysql_data()

# Gabungkan berdasarkan kolom 'id' dan 'Class'
df_merged = pd.merge(df_mysql, df_postgresql, on=["id", "class"], how="inner")

# Ambil data fitur numerik
feature_columns = ["petal_length", "petal_width", "sepal_length", "sepal_width"]
data_values = df_merged[feature_columns].values

In [10]:
# Inisialisasi model LOF
clf = LocalOutlierFactor(n_neighbors=80)
label = clf.fit_predict(data_values)

# Tambahkan hasil label ke dataframe
df_merged["outlier_label"] = label

# Cetak hasil dengan ID dan class
display(HTML(df_merged.to_html(index=False)))

num_outliers = (label == -1).sum()
print(f"\nJumlah outlier: {num_outliers}")

print("\n")

outliers = df_merged[df_merged["outlier_label"] == -1]
print("\nData Outlier:")
display(HTML(outliers.to_html(index=False)))

id,class,petal_length,petal_width,sepal_length,sepal_width,outlier_label
1,Iris-setosa,1.4,0.2,5.1,3.5,1
2,Iris-setosa,14.0,2.0,40.9,30.0,-1
3,Iris-setosa,1.3,0.2,4.7,3.2,1
4,Iris-setosa,1.5,0.2,4.6,3.1,1
5,Iris-setosa,1.4,0.2,5.0,3.6,1
6,Iris-setosa,1.7,0.4,5.4,3.9,1
7,Iris-setosa,1.4,0.3,4.6,3.4,1
8,Iris-setosa,1.5,0.2,5.0,3.4,1
9,Iris-setosa,1.4,0.2,4.4,2.9,1
10,Iris-setosa,1.5,0.1,4.9,3.1,1



Jumlah outlier: 1

Data Outlier:


id,class,petal_length,petal_width,sepal_length,sepal_width,outlier_label
2,Iris-setosa,14.0,2.0,40.9,30.0,-1
